In [1]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import csv
import re

In [2]:
tm_links = "http://www.trustile.com/products/panel-doors/tm-series"
soup = BeautifulSoup(urlopen(tm_links).read())
    
tmList = []
for a in soup.findAll('a'):
    try:
        urlClass = a.get('class')
        if urlClass[0]=="gridview":
            tmList.append(a.get('href'))
            
    except:
        pass

In [3]:
base_url = "http://www.trustile.com/"
def make_url(str):
    return base_url + str

In [4]:
def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html)

In [5]:
def get_doorName(soup):
    return soup.h1.string

In [6]:
def get_pageFields(soup):
    return soup.findAll('fieldset')

In [7]:
def get_images(field):
    # get all types of images of this door model
    div = field.find('div',attrs={"class":"doorthumbs"})
    images = []
    for a in div.findAll('a'):
        aImage = {}
        aImage["type"] = a.get('alt')
        aImage["url"] = a.get('href')
        images.append(aImage)
    return images
            

In [8]:
# detail information on the right of the page
def get_details(field):
    div = field.find('div',attrs={"class":"doordetails"})
    details = {}
    for p in div.findAll('p'):
        try:
            ifDetail = p.get('class')[2]
            if ifDetail == "product-detail":           
                category = p.span
                if category.string == u'Series:\xa0':
                    details['Series'] = p.a.string
    
                if category.string == u'Application:\xa0':
                    stri = p.get_text()
                    app = re.search(u'(?<=\xa0).*',stri).group()
                    details['Application'] = app
    
                if category.string == u'Thickness:\xa0':
                    details['Thickness'] = p.findAll('span')[1].get_text()
    
                if category.string == u'Construction:\xa0':
                    construction = []
                    for child in p.findAll('span')[1].children:
                        construction.append(child.string)
                    details['Construction'] = construction
                if category.string == u'Available Fire Rating:\xa0':
                    fireRating = []
                    for img in p.findAll('span')[1].findAll('img'):
                        aRating = {}
                        aRating["type"] = img.get('alt')
                        aRating["url"] = img.get('src')
                        fireRating.append(aRating)
                    details['Fire'] = fireRating
        except:
            print p.get_text()
    return details

In [9]:

def get_profiles(field):
    profiles = []
    for a in field.findAll('a'):
        profiles.append(a.get('title'))

    return profiles

In [10]:
# TM series
# 
outFile = open("TrustileTM_Doors.csv", "wb")
writer = csv.writer(outFile)
writer.writerow(["","Name","Images","Series","Application","Construction","Thickness","Fire Rating","Profiles Options","Other Options"])

for i in range(len(tmList)):
    soup = make_soup(make_url(tmList[i]))
    
    name = get_doorName(soup)
    
    options = []
    fields = get_pageFields(soup)
    for field in fields:
        if field.legend.span.string == "Overview":
            images = get_images(field)
            
            details = get_details(field)
            try:
                fireRating = details["Fire"]
            except:
                fireRating = ""
        
        if field.legend.span.string == "Profiles":
            profiles = get_profiles(field)
        
        # tab3: wood species
        if field.legend.span.string == "Wood Species":
            options.append('Wood')
    
        # tab4: glass options
        if field.legend.span.string == "Glass":
            options.append('Glass')
        
        # tab5: leather options
        if field.legend.span.string == "Leather":
            options.append('Leather')
    
        # tab6: Metal options
        if field.legend.span.string == "Metal":
            options.append('Metal')
    
#     print ([i+1, name, images, 
#                      details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
#                     options])
    writer.writerow([i+1, name, images, 
                     details["Series"], details["Application"], details["Construction"], details["Thickness"], fireRating,
                     profiles,
                     options])
outFile.close()

Note: Available only with leather panels.
Note: Available only with leather panels.


In [11]:
### store TM series profile options

In [20]:
soup = make_soup("http://www.trustile.com/products/panel-doors/tm-series/tm13270")
fields = get_pageFields(soup)
for field in fields:
    if field.legend.span.string == "Profiles":
        for a in field.findAll('a'):
            print a.get("title")
            print a.get("href")

1/8" Radius Reveal
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/reveal-radius.png?itok=rRUvutpO
1/4" Kerf Cut Reveal
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/reveal-kerf.png?itok=ksDn2H-G
Square Stick (SS) Profile
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/modern-square-stick-profile_0.png?itok=Mh9ty-OW
One-Step (OS) Profile
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/modern-one-step-profile_0.png?itok=5gohGYuE
Square Stick (SS) Sash
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/modern-square-stick-sash.png?itok=KTzXRhI4
One Step (OS) Sash
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/modern-one-step-sash.png?itok=jQ4-7vHp
Square Stick (SS) Compression Fit Sash
http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/modern-square-stick-compression-sash.png?itok=Ihd

In [21]:
soup = make_soup("http://www.trustile.com/products/panel-doors/tm-series/tm13270")
fields = get_pageFields(soup)

profiles = open("TrustileTM_profiles.csv","wb")
w = csv.writer(profiles)
w.writerow(["Type","Name","URL"])
for field in fields:
    if field.legend.span.string == "Profiles":
        for a in field.findAll('a'):
            w.writerow(["",a.get('title'),a.get('href')])
profiles.close()

In [ ]:
#### in process codes

In [17]:
soup = make_soup("http://www.trustile.com/products/panel-doors/tm-series/tm13270")
fields = get_pageFields(soup)
for field in fields:
    if field.legend.span.string == "Profiles":
        print field.prettify()

<fieldset class="collapsible collapsed group-door-profiles field-group-htab form-wrapper">
 <legend>
  <span class="fieldset-legend">
   Profiles
  </span>
 </legend>
 <div class="fieldset-wrapper">
  <h3>
   Reveal Options
  </h3>
  <table class="views-view-grid cols-2">
   <tbody>
    <tr>
     <td>
      <div class="field-profile-image">
       <div class="field-items">
        <div class="field-item odd">
         <a class="colorbox" href="http://www.trustile.com/sites/trustile.com/files/styles/large/public/profiles/reveal-radius.png?itok=rRUvutpO" rel="gallery-field_profile_image" title='1/8" Radius Reveal'>
          <img alt="" src="http://www.trustile.com/sites/trustile.com/files/styles/scale_25/public/profiles/reveal-radius.png?itok=Pkb2wdqc" title=""/>
         </a>
        </div>
       </div>
      </div>
      <span class="caption">
       1/8" Radius Reveal
      </span>
      <span class="caption note">
      </span>
     </td>
     <td>
      <div class="field-profile-i